In [148]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [149]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
movie_gross_df = pd.read_csv('data/bom.movie_gross.csv')
name_df = pd.read_csv('data/imdb.name.basics.csv')
title_akas_df = pd.read_csv('data/imdb.title.akas.csv')
title_basics_df = pd.read_csv('data/imdb.title.basics.csv')
title_crew_df = pd.read_csv('data/imdb.title.crew.csv')
title_principals_df = pd.read_csv('data/imdb.title.principals.csv')
title_ratings_df = pd.read_csv('data/imdb.title.ratings.csv')
rt_movie_info_df = pd.read_csv('data/rt.movie_info.tsv', delimiter='\t')
rt_reviews_df = pd.read_csv('data/rt.reviews.tsv', encoding='windows-1254', delimiter='\t')
tmdb_movies_df = pd.read_csv('data/tmdb.movies.csv')
tn_budgets_df = pd.read_csv('data/tn.movie_budgets.csv')

In [151]:
from data_cleaner import clean_movie_budgets
from data_cleaner import clean_tmdb_movies

In [152]:
tmdb_movies_clean_df = clean_tmdb_movies(tmdb_movies_df)
tn_budgets_df = clean_movie_budgets(tn_budgets_df)

In [153]:
tmdb_movies_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[adventure, fantasy, family]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[fantasy, adventure, animated, family]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[adventure, action, sci-fi]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[animated, comedy, family]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[action, sci-fi, adventure]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [154]:
tn_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,domestic_production_ratio,worldwide_production_ratio,worldwide_domestic_ratio
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,1.789430,6.532577,3.650648
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,0.587101,2.546673,4.337705
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,0.122178,0.427892,3.502201
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.388403,4.243841,3.056636
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,1.956408,4.153696,2.123124
